## Step 15: 복잡한 계산 그래프 (이론 편)

지금까지 우리는 일직선의 계산 그래프만을 다루어보았다. 그러나 모든 계산이 이러하리란 법은 없다.

현재의 DeZero에서도 복잡한 계산의 순전파는 정상적으로 수행할 수 있다. 하지만 역전파는 그렇지 못하다. 이는 역전파의 순서를 고려하는 부분이 없기 때문인데, 이를 해결해보도록 하겠다.

### 15.1 역전파의 올바른 순서

<img width="80%" alt="image" src="https://user-images.githubusercontent.com/76294398/178418977-0968ea00-f8ed-4b8f-97b0-1c41d8251b1e.png">

위와 같이 중간에 분기했다가 다시 합류하는 계산 그래프가 있다고 가정해보자. 이는 비교적 간단한 모습이지만 현재의 DeZero에서는 이조차 역전파를 제대로 수행하지 못한다. 이는 변수 a의 미분이 계산되려면 b의 미분과 c의 미분이 모두 전달되어야 하기 때문인데, 현재는 b와 c 중 하나만 완료되면 그대로 a의 미분 계산이 시작되어 버리기 때문에 문제가 발생한다.

### 15.2 현재의 DeZero

현재의 DeZero는 `funcs.pop()`과 `funcs.append(x.creator)`에 의해 함수 실행 순서가 결정된다. 이에 따라 위 그림과 같은 계산 그래프에서 순서 문제가 발생하는 것이다.

가장 먼저 funcs 리스트에 D가 추가되어 [D] 상태로 시작한다. 여기서 `funcs.pop()`에 의해 D가 꺼내지고, D의 creator인 B와 C가 funcs 리스트에 추가된다. ([B,C])


그 다음, 다시 `funcs.pop`에 의해 C가 꺼내진 후 C의 creator A가 funcs 리스트에 추가된다. 바로 이 시점에서 문제가 생기는데, 본래대로라면 다음 `funcs.pop()`에서 B가 꺼내져야 하지만, funcs 리스트가 [B,A]가 되는 바람에 A가 잘못된 순서로 꺼내지고 계산이 수행된다.

이와 같은 방식으로 A → B 순으로 역전파가 진행된 후, B의 creator인 A가 다시 리스트에 추가되며 두 번 계산되는 문제가 발생한다.

### 15.3 함수 우선순위

이러한 문제가 발생하는 이유는 '함수 간 우선순위'를 설정하지 않았기 때문이다. 지금까지는 리스트에서 순서를 고려하지 않고 마지막 원소를 단순히 꺼내서 계산하였는데, 이때문에 위와 같은 문제가 발생한 것이다.

그렇다면, 우선순위를 지정해주면 될 일이다. 하지만, 어떤 순서로 어떻게 지정해줘야 할까?

첫 번째로, 주어진 계산 그래프를 분석하여 알아내는 방법이 있다. 가령 위상 정렬(Topological sort) 알고리즘을 사용하면 노드의 연결 방법을 기초로 하여 노드를 정렬할 수 있다. 이렇게 정렬된 그래프를 우선순위로써 사용할 수 있다. 하지만 이는 위상 정렬 알고리즘을 추가해야 한다는 점에서 조금 번잡한 감이 있다.

두 번째 방식은 세대를 나누는 것이다. 우리는 순전파 진행 시 변수와 함수 사이 creator를 설정하였다. 이때 이 '창조자-피조물' 관계를 기준으로 세대(generation)를 기록하는 방법을 사용하면 보다 간편하게 우선순위를 생성할 수 있다.

<img width="80%" alt="image" src="https://user-images.githubusercontent.com/76294398/178425088-bb813f04-36a6-4ed7-b49c-fb2aec255560.png">

위처럼 관계를 기준으로 세대를 생성한 후 세대 수가 큰 쪽부터 처리하도록 한다면 문제가 해결될 것이다.

이를 다음 단계에서 코드로 구현해보도록 하겠다.